In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
import tensorflow as tf


import keras.backend as K

Using TensorFlow backend.


In [2]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train.txt", "EmoContext")
df.head(20)

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others
5,5,I'm a dog person,youre so rude,Whaaaat why,others
6,6,So whatsup,Nothing much. Sitting sipping and watching TV....,What are you watching on tv?,others
7,7,Ok,ok im back!!,"So, how are u",others
8,8,Really?,really really really really really,Y saying so many times...i can hear you,others
9,9,Bay,in the bay,😘 love you,others


In [3]:
df_test = functions.parse_file(r"testwithoutlabels.txt", "EmoContext")

In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append(" {} fullstop {} fullstop {} fullstop".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
full_text_data = []
for idx,row in df_test.iterrows():
    full_text_data.append(" {} fullstop {} fullstop {} fullstop".format(row['turn1'], row['turn2'], row['turn3']))
full_text_data.extend(text_data)

In [6]:
emoticons_map = {
    '😘': ' emoticon',
    '😍': ' happyemoticon',
    '😁': ' happyemoticon',
    '😭': ' sademoticon',
    '😑': ' sademoticon',
    '😻': ' happyemoticon',
    '😂': ' happyemoticon',
    '👍': ' emoticon',
    '😀': ' happyemoticon',
    ':D': ' happyemoticon',
    '🙂':  ' happyemoticon',
    '<3': ' happyemoticon',
    '😓' : ' sademoticon',
    '😒' : ' angryemoticon',
    '😈' : ' emoticon',
    '👿' : ' angryemoticon',
    '🖑' : ' happyemoticon',
    '😾' : ' emoticon',
    '😠' : ' angryemoticon',
    '👻' : ' emoticon',
    ':(' : ' sademoticon',
    ':)' : ' happyemoticon',
    'xD' : ' happyemoticon',
    '💔' : ' sademoticon',
    '😥' : ' emoticon',
    '😞' : ' sademoticon',
    '😤' : ' angryemoticon',
    '😃' : ' happyemoticon',
    '😦' : ' sademoticon',
    ':3' : ' emoticon',
    '😼' : ' emoticon',
    '😏' : ' happyemoticon',
    '😱' : ' sademoticon',
    '😬' : ' sademoticon',
    '🙁' : ' sademoticon',
    '</3' : ' sademoticon',
    '😺' : ' happyemoticon',
    '😣' : ' angryemoticon',
    '😢' : ' sademoticon',
    '😆' : ' happyemoticon',
    '😄' : ' happyemoticon',
    '😅' : ' happyemoticon',
    ':-)' : ' happyemoticon',
    '😊' : ' happyemoticon',
    '😕' : ' sademoticon',
    '😽' : ' happyemoticon',
    '🙀' : ' angryemoticon',
    '🤣' : ' happyemoticon',
    '🤐' : ' emoticon',
    '😡' : ' sademoticon',
    '👌' : ' happyemoticon', 
    '😮' : ' emoticon',
    '❤️' : ' happyemoticon',
    '🙄' : ' happyemoticon',
    '😿' : ' sademoticon',
    '😉' : ' happyemoticon',
    '😋' : ' happyemoticon',
    '😐' : ' emoticon',
    '😹' : ' happyemoticon',
    '😴' : ' sademoticon',
    '💤' : ' emoticon',
    '😜' : ' happyemoticon',
    '😇' : ' happyemoticon',
    '😔' : ' sademoticon',
    '😩' : ' sademoticon',
    '❤' : ' happyemoticon',
    '😲' : ' emoticon',
    '😫' : ' sademoticon',
    '😳' : ' sademoticon',
    '😰' : ' sademoticon',
}

In [7]:
bad_words_df = pd.read_csv('bad-words.csv')
#bad_words_df['jigaboo']
bad_words = list(bad_words_df['jigaboo'])
bad_words.sort(key = lambda s: len(s))
bad_words = bad_words[::-1]

In [8]:
for i in range(0, len(text_data)):
    for k in emoticons_map.keys():
        if text_data[i].find(k) >= 0:
            text_data[i] = text_data[i].replace(k, emoticons_map[k])
    for bw in bad_words:
        bw = " {} ".format(bw)
        if text_data[i].find(bw) >= 0:
            text_data[i] = text_data[i].replace(bw, ' badword ')
            

In [9]:
for i in range(0, len(full_text_data)):
    for k in emoticons_map.keys():
        if full_text_data[i].find(k) >= 0:
            full_text_data[i] = full_text_data[i].replace(k, emoticons_map[k])
    for bw in bad_words:
        bw = " {} ".format(bw)
        if full_text_data[i].find(bw) >= 0:
            full_text_data[i] = full_text_data[i].replace(bw, ' badword ')
            

In [10]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [11]:
NR_WORDS = 17000

In [12]:
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(full_text_data)

In [13]:
X = tokenizer.texts_to_sequences(text_data)
X = pad_sequences(X)

In [14]:
words = {"others": 0, "angry": 1, "sad":2, "happy": 3}

def one_hot_vector(word):
    y = [0,0,0,0]
    y[words[word]] = 1
    return y

Y = []

for idx,row in df.iterrows():
    Y.append(one_hot_vector(row['label']))

Y = np.array(Y)

#### model

In [15]:
embed_dim = 256
lstm_out = 128
batch_size = 128

model = Sequential()
model.add(Embedding(NR_WORDS, embed_dim,input_length = X.shape[1]))
model.add(Bidirectional(LSTM(lstm_out)))
model.add(Dropout(0.3))
model.add(Dense(4,activation='softmax'))
adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 160, 256)          4352000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
Total params: 4,747,268
Trainable params: 4,747,268
Non-trainable params: 0
_________________________________________________________________


In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0, random_state = 42)

In [ ]:
#model.fit(X_train, Y_train, batch_size = 128, nb_epoch = 3, verbose = 1)

In [ ]:
K.set_value(model.optimizer.lr, 0.0001)

In [18]:
from sklearn.model_selection import KFold
# prepare cross validation
kfold = KFold(n_splits=4)
Y_train = np.array(Y_train)
# enumerate splits
for train, validation in kfold.split(X_train):
    history = model.fit(X_train[train], Y_train[train],
                    validation_data=(X_train[validation], Y_train[validation]),
                    epochs=1, verbose=1, batch_size=64)

Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 516s 23ms/step - loss: 0.5697 - acc: 0.7898 - f1: 0.7432 - val_loss: 0.3861 - val_acc: 0.8618 - val_f1: 0.8484
Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 501s 22ms/step - loss: 0.3440 - acc: 0.8786 - f1: 0.8643 - val_loss: 0.2988 - val_acc: 0.8890 - val_f1: 0.8752
Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 541s 24ms/step - loss: 0.2734 - acc: 0.9062 - f1: 0.8939 - val_loss: 0.2908 - val_acc: 0.8958 - val_f1: 0.8789
Train on 22620 samples, validate on 7540 samples
Epoch 1/1
22620/22620 [==============================] - 509s 22ms/step - loss: 0.2354 - acc: 0.9183 - f1: 0.9094 - val_loss: 0.1973 - val_acc: 0.9313 - val_f1: 0.9224


#### Evaluation

In [ ]:
ev_result = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.3f" % (ev_result[0]))
print("Validation Accuracy: %.3f" % (ev_result[1]))
print("F1 score: %.3f" % (ev_result[2]))

In [19]:
model_json = model.to_json()
with open("simple_model.json", "w") as outfile:
    outfile.write(model_json)
model.save_weights("simple_model.h5")

#### For submission

In [20]:
df_test = functions.parse_file(r"testwithoutlabels.txt", "EmoContext")
df_test.head()

,id,turn1,turn2,turn3
0,0,Hmm,What does your bio mean?,I don’t have any bio
1,1,What you like,very little things,Ok
2,2,Yes,How so?,I want to fuck babu
3,3,what did you guess,what what,fuck
4,4,We ?,of course we will!,What gender movies you like??


In [21]:
text_data = []

for idx,row in df_test.iterrows():
    text_data.append(" {} fullstop {} fullstop {} fullstop".format(row['turn1'], row['turn2'], row['turn3']))
    
for i in range(0, len(text_data)):
    for k in emoticons_map.keys():
        if text_data[i].find(k) >= 0:
            text_data[i] = text_data[i].replace(k, emoticons_map[k])
    for bw in bad_words:
        bw = " {} ".format(bw)
        if text_data[i].find(bw) >= 0:
            text_data[i] = text_data[i].replace(bw, ' badword ')    
    
X_test = tokenizer.texts_to_sequences(text_data)
X_test = pad_sequences(X_test, maxlen = X.shape[1])

In [22]:
res = model.predict(X_test, batch_size=128, verbose=1)

5509/5509 [==============================] - 24s 4ms/step


In [23]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [24]:
results = []
for r in res:
    results.append(softmax_convert(r))

In [25]:
df_test['label'] = results

In [26]:
df_test.head(30)

,id,turn1,turn2,turn3,label
0,0,Hmm,What does your bio mean?,I don’t have any bio,others
1,1,What you like,very little things,Ok,others
2,2,Yes,How so?,I want to fuck babu,sad
3,3,what did you guess,what what,fuck,others
4,4,We ?,of course we will!,What gender movies you like??,others
5,5,Where are you now?,At home just about to have breakfast...,what are you eating?,others
6,6,That was a joke btw...,it was,Yes 😁,happy
7,7,Who d hell s he,johnny depp...duh?,Who she,others
8,8,"yes, good advice",best advice ...,i great thx,others
9,9,Nice to meet u,"Hi, nice to meet you too! 😸😂",😁,happy


In [27]:
df_test.to_csv("test.txt",index=False , sep="\t")

In [ ]:
dir(tokenizer)

In [ ]:
len(tokenizer.word_counts)